# SETI_DEMO: Toy multi-band signal → Evolutionary Game analysis

**Caveat:** This is a *toy* demo intended to exercise the pipeline on multi-channel signals.
It is **not** evidence of life, intelligence, or anything beyond synthetic coordination.
Use only as a methodological dry run with rigorous nulls.


In [ ]:
import sys, os, numpy as np, pandas as pd, matplotlib.pyplot as plt
sys.path.append('/mnt/data')
from gameify_timeseries import info_gain_payoffs, estimate_A_from_series, find_ESS
print('Imports OK')

## 1) Simulate a toy "engineered" allocation across frequency bands
We create `N=8` bands over `T=600` steps. A hidden controller mixes `K=3` transmit archetypes under changing noise, with mild cyclic dynamics.


In [ ]:
def simulate_bands(N=8, K=3, T=600, seed=0):
    rng = np.random.default_rng(seed)
    # True archetypes S_true (nonnegative, column-normalized)
    S = rng.random((N, K))
    S /= (np.linalg.norm(S, axis=0, keepdims=True) + 1e-12)
    # Hidden replicator-like dynamics in x(t)
    A_true = np.array([[ 0.0,  0.3, -0.2],
                       [-0.3,  0.0,  0.25],
                       [ 0.2, -0.25, 0.0]])  # skew → cycles
    x = np.full(K, 1.0/K)
    X = np.zeros((N, T))
    noise_env = rng.standard_normal((N, T)) * 0.3  # band-wise exogenous noise
    for t in range(T):
        # band power allocation + env noise (rectified)
        X[:, t] = np.maximum(S @ x + 0.15*noise_env[:, t], 0)
        # update x via one Euler step of replicator on A_true with small diffusion
        u = A_true @ x
        u -= u.mean()  # centered payoff
        dx = x * u
        x = x + 0.1*dx + 0.01*rng.random(K)
        x = np.maximum(x, 1e-8); x = x / x.sum()
    return S, X

S_true, X = simulate_bands()
N, T = X.shape; K = 3
bands = [f'B{i+1}' for i in range(N)]
print('X shape:', X.shape)

## 2) Strategy basis via simple NMF on band powers
We learn `S_hat` from `X` (nonnegative) and normalize columns.


In [ ]:
def nmf_multiplicative(V, r=3, iters=400, seed=1):
    rng = np.random.default_rng(seed)
    N, T = V.shape
    W = np.maximum(rng.random((N, r)), 1e-6)
    H = np.maximum(rng.random((r, T)), 1e-6)
    for _ in range(iters):
        WH = W @ H + 1e-12
        H *= (W.T @ (V / WH)) / (W.T @ np.ones_like(V) + 1e-12)
        WH = W @ H + 1e-12
        W *= ((V / WH) @ H.T) / (np.ones_like(V) @ H.T + 1e-12)
        W = np.maximum(W, 1e-12); H = np.maximum(H, 1e-12)
    return W, H

S_hat, H = nmf_multiplicative(X, r=K, iters=300)
S_hat = S_hat / (np.linalg.norm(S_hat, axis=0, keepdims=True) + 1e-12)
print('S_hat shape:', S_hat.shape)

## 3) Define a target and compute **information-gain payoffs**
We synthesize a scalar target `J` (detection score proxy) as a weighted sum of two bands plus noise, then compute rolling LOFO ablations.


In [ ]:
rng = np.random.default_rng(2)
w = np.zeros(N); w[1] = 0.7; w[5] = -0.5
J = X.T @ w + 0.2*rng.standard_normal(T)
from gameify_timeseries import info_gain_payoffs
v = info_gain_payoffs(X, J, ridge=1e-2, window=60)
print('v shape:', v.shape)

## 4) Fit strategy-level operator `A` and search for ESS

In [ ]:
from gameify_timeseries import estimate_A_from_series, find_ESS
est = estimate_A_from_series(S_hat, X, v, k=K, lambda_=1e-2)
A = est['A']
print('R^2:', round(est['R2'], 3))
ess = [r for r in find_ESS(A, tol=1e-8, max_support=K) if r['is_ess']]
print('ESS count:', len(ess))

## 5) Null test: phase-scramble each band
We preserve each band's power spectrum but destroy cross-band phase coordination; the pipeline should weaken.


In [ ]:
def phase_scramble(ts, rng):
    T = ts.size
    f = np.fft.rfft(ts)
    phases = rng.uniform(0, 2*np.pi, size=f.shape)
    phases[0] = 0.0
    if T % 2 == 0:
        phases[-1] = 0.0
    f_scr = np.abs(f) * np.exp(1j*phases)
    return np.fft.irfft(f_scr, n=T).real

rng = np.random.default_rng(42)
X_ps = np.zeros_like(X)
for i in range(N):
    X_ps[i] = phase_scramble(X[i], rng)
S_ps, _ = nmf_multiplicative(X_ps, r=K, iters=300, seed=3)
S_ps = S_ps / (np.linalg.norm(S_ps, axis=0, keepdims=True) + 1e-12)
v_ps = info_gain_payoffs(X_ps, J, ridge=1e-2, window=60)
est_ps = estimate_A_from_series(S_ps, X_ps, v_ps, k=K, lambda_=1e-2)
A_ps = est_ps['A']
ess_ps = [r for r in find_ESS(A_ps, tol=1e-8, max_support=K) if r['is_ess']]
print('R^2 (phase-scrambled):', round(est_ps['R2'], 3), 'ESS:', len(ess_ps))

## 6) Quick plots

In [ ]:
plt.figure(figsize=(7,3))
for i in range(K):
    plt.plot(est['Xk'][i], label=f'x_{i+1}')
plt.title('Inferred strategy mixture x(t)'); plt.legend(); plt.show()

As = 0.5*(A + A.T); Aa = 0.5*(A - A.T)
print('||A_s||_F, ||A_a||_F =', float(np.linalg.norm(As)), float(np.linalg.norm(Aa)))

### Conclusion (toy)
- If the pipeline shows stronger fit/ESS on the synthetic "engineered" signal than on the phase-scrambled surrogate, the method acts as a **detector of coordinated trade-offs**.
- **This does not imply life or intent**; real SETI would require far stronger evidence and instrument/astrophysical nulls.
